# Работа модели для матчинга резюме и вакансий портала "Работа в России":

In [ ]:
#Установим библиотеку, разработанную ЦПУР 
!git clone https://github.com/CAG-ru/rostrud-ml.git

После скачивания корневая папка должна быть переименована в rostrud_ml

## Предварительные действия
У этого скрипта есть несколько зависимостей, их можно поставить через pip. При запуске блока кода ниже всё поставится само.

Если в какой-то момент выполнения кода вы увидите ошибку, что вам не хватает некого питоновского пакета, вы можете дописать его название в новую строчку файла requirements.txt и снова запустить этот блок.

In [ ]:
#Установка окружения
!pip install -r rostrud_ml/requirements.txt

В файле rostrud_ml/utils/all_tables_names.yml указать путь к директории, куда будут скачиваться и парситься файлы с резюме, вакансиями (сейчас там "workdir").

В файле rostrud-ml/utils/config_to_bd_example.yml указать ваши настройки для подключения к БД и поменять название файла на config_to_bd.yml

## Загрузка в БД необходимых данных

In [ ]:
#Установка модулей
from rostrud_ml.process.adding_tables_psycopg import AddingDataPsycopg
from rostrud_ml.process.renewal import Renewal

In [ ]:
#Cоздание таблиц в БД
db = AddingDataPsycopg()
for table in ['curricula_vitae', 'workexp', 'vacancies', 'professions', 'industries', 'regions']:
    db.create_table(table, 'project_trudvsem') #'project_trudvsem' - название схемы в вашей БД
    
db.conn.close()

Взять ссылки на файлы с данными с ftp поратала https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/:

Из директорий
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-cv/
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-vacancy/ нужно выбрать подходящие файлы (имеющие в наименовании дату) - один файл резюме и один вакансий.

Подобным образом выбрать ссылки для данных о профессиях, сферах занятости, регионах из соответствующих директорий:
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-professions/ 
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-industries/
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-regions/

Для каждого типа, вставив нужную ссылку, запустить следующие ячейки:
В примере: тип "резюме" и самый первый файл резюме 2018г. Необходимо поменять ссылку на наиболее актуальный файл.

In [ ]:
#создать экземпляр класса с нужными параметрами: 
#указать тип файла ('cvs' для резюме), 
#а также путь к тому xml-файлу, который содержит нужные данные
#ссылку ниже необходимо заменить
ren = Renewal('cvs', 'https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-cv/data-20180102T024002-structure-20161130T143000.xml.gz')

#последовательно вызываются функции для скачивания, парсинга, обработки, загрузки в БД и удаления первоначальных файлов
ren.download()
ren.parse_update()
ren.delete()

#данные об опыте работы, в таблицу workexp будут также загружены при выполнении этой ячейки

In [ ]:
#вставьте правильную ссылку 
ren = Renewal('vacancies', '')
ren.download()
ren.parse_update()
ren.delete()

In [ ]:
#вставьте правильную ссылку 
ren = Renewal('professions', '')
ren.download()
ren.parse_update()
ren.delete()

In [ ]:
#вставьте правильную ссылку 
ren = Renewal('industries', '')
ren.download()
ren.parse_update()
ren.delete()

In [ ]:
#вставьте правильную ссылку 
ren = Renewal('regions', '')
ren.download()
ren.parse_update()
ren.delete()

## Обучение модели

In [ ]:
"""Создаем новые таблицы лемм вакансий и резюме для матчинга
(Для первой загрузки записей в таблицы для матчинга и обучения модели)"""
db = AddingDataPsycopg()
db.create_table('for_matching_cv', 'project_trudvsem') #'project_trudvsem' название вашей схемы в БД
db.create_table('for_matching_vac', 'project_trudvsem') #'project_trudvsem' название вашей схемы в БД
id_cv_list = []
id_vac_list = []

"""Получаем новые записи, которых нет в таблицах для матчинга"""
db = AddingDataPsycopg()
df_cv = db.get_table_as_df_join('curricula_vitae', 'get_df_join')
df_vac = db.get_table_as_df_join('vacancies', 'get_df_join')

"""Подготавливаем записи для загрузки в таблицы для матчинга"""
prepare_cv = PrepareData(russian_stopwords_and_demands, w2v_model)
prepare_vac = PrepareData(russian_stopwords_and_respons, w2v_model)
df_cv = prepare_cv.prepare_df(df_cv, id_cv_list, 'curricula_vitae')
df_vac = prepare_vac.prepare_df(df_vac, id_vac_list, 'vacancies')

"""Загружаем новые записи в таблицы для матчинга"""
db.write_to_sql(df_cv, 'for_matching_cv', 'project_trudvsem') #'project_trudvsem' название вашей схемы в БД
db.write_to_sql(df_vac, 'for_matching_vac', 'project_trudvsem') #'project_trudvsem' название вашей схемы в БД

"""Обучение модели"""
text_lemmas = prepare_vac.lemmas_to_list(df_cv, df_vac)
prepare_vac.train_model(text_lemmas)

## Повторное обучение модели

In [ ]:
"""Получаем новые записи, которых нет в таблицах для матчинга"""
db = AddingDataPsycopg()
df_cv = db.get_table_as_df_join('curricula_vitae', 'get_df_join')
df_vac = db.get_table_as_df_join('vacancies', 'get_df_join')

"""Шаг для загрузки лемм записей в таблицы для матчинга"""
df = db.get_table_as_df('identifier', 'for_matching_cv', 'project_trudvsem') #'project_trudvsem' название вашей схемы в БД
id_cv_list = df['identifier'].tolist()
df = db.get_table_as_df('identifier', 'for_matching_vac', 'project_trudvsem') #'project_trudvsem' название вашей схемы в БД
id_vac_list = df['identifier'].tolist()

"""Подготавливаем записи для загрузки в таблицы для матчинга"""
prepare_cv = PrepareData(russian_stopwords_and_demands, w2v_model)
prepare_vac = PrepareData(russian_stopwords_and_respons, w2v_model)
df_cv = prepare_cv.prepare_df(df_cv, id_cv_list, 'curricula_vitae')
df_vac = prepare_vac.prepare_df(df_vac, id_vac_list, 'vacancies')

"""Загружаем новые записи в таблицы для матчинга"""
db.write_to_sql(df_cv, 'for_matching_cv', 'project_trudvsem') #'project_trudvsem' название вашей схемы в БД
db.write_to_sql(df_vac, 'for_matching_vac', 'project_trudvsem') #'project_trudvsem' название вашей схемы в БД

"""Удаляем не актуальные записи, которых нет в исходных таблицах резюме и вакансий"""
db.delete_strings('identifier', 'id_cv', 'for_matching_cv', 'curricula_vitae', 'project_trudvsem') #'project_trudvsem' название вашей схемы в БД
db.delete_strings('identifier', 'identifier', 'for_matching_vac', 'vacancies', 'project_trudvsem') #'project_trudvsem' название вашей схемы в БД
db.conn.close()

"""Проверяем количество обновленных записей и дообучаем модель при условии большого объема 
новых записей(если загрузка первичная пропускаем)"""
print('Строк в таблице резюме ' + str(len(id_cv_list)) + ' добавлено строк ' + str(df_cv.shape[0]))
print('Строк в таблице вакансий ' + str(len(id_vac_list)) + ' добавлено строк ' + str(df_vac.shape[0]))
if (len(id_cv_list) + len(id_vac_list)) == 0:
    print('Дообучение модели не требуется')
else:
    difference = ((df_cv.shape[0] + df_vac.shape[0]) / (len(id_cv_list) + len(id_vac_list)))
    if difference <= 0.4:
        print('Дообучение модели не требуется:', difference)
    else:
        text_lemmas = prepare_vac.lemmas_to_list(df_cv, df_vac)
        prepare_vac.retrain_model(text_lemmas)

## Процедура подбора резюме и вакансий

In [ ]:
"""Загружаем предобученную модель и  стандар скеллер для фич"""
w2v_model = Word2Vec.load('w2v_model_retrain.model')
w2v_model.init_sims(replace = True)
scaler = pickle.load(open('rostrud_ml/matching/text_matching/scaler_short.pkl','rb'))
similarity = CosineSimilarityW2v(w2v_model, scaler)

"""Подберем вакансии для резюме, указываем id резюме
и количество вакансий для вывода
ВАРИАНТ чистовой  с выводом строчек резюме и вакансий"""
matching = MatchingData(10, similarity)
df_cv, df_vac = matching.trudvsem_profile_w2v_to_df_cv('00624890-fc26-11eb-81cd-6db06c9eaf56')
print(df_cv)
print(df_vac) 

"""Подберем резюме для вакансии, указываем id вакансии
и количество резюме для вывода
ВАРИАНТ чистовой с выводом строчек резюме и вакансий"""
df_vac, df_cv = matching.trudvsem_profile_w2v_to_df_vac('00118432-3152-11ec-a785-bf2cfe8c828d')
print(df_vac)
print(df_cv) 